https://www.youtube.com/watch?v=_HDMCCLlGl4

In [1]:
from os import listdir
from os.path import isfile, join
import pathlib
import pandas as pd

In [2]:
tac_path = 'C:/Users/peter/Desktop/Spring_2020/Machine_Learning/project/clean_tac'

In [3]:
tac_files = [f for f in listdir(tac_path) if isfile(join(tac_path, f))]

In [4]:
#read in tac data, add 3 zeros to each to match size of accelerometer timestamp ints
tac = pd.DataFrame(columns = ['timestamp','TAC_Reading','pid'])

for i in range(len(tac_files)):
    tac_temp = pd.read_csv(tac_path + '/' + tac_files[i])
    tac_temp['pid'] = tac_files[i][0:6]
    tac = tac.append(tac_temp)
    
tac['time'] = pd.to_numeric(tac['timestamp'].astype(str) + '000')
tac = tac.drop('timestamp', axis = 1)

In [5]:
#acc data is in same path as this notebook
#read acc data
acc = pd.read_csv('all_accelerometer_data_pids_13.csv')
ids = acc['pid'].unique()

In [6]:
comb = pd.DataFrame(columns = pd.merge_asof(acc[acc['pid'] == ids[0]],tac[tac['pid'] == ids[0]], on = 'time').columns)

for i in range(len(ids)):
    comb = comb.append(pd.merge_asof(acc[acc['pid'] == ids[i]],tac[tac['pid'] == ids[i]], on = 'time', direction = 'nearest'))

In [7]:
#dropping two instances where accelerometer output is zero for all readings
comb = comb[comb['time'] != 0]

#shouldn't be any NAs but whatever
comb = comb.dropna(axis = 0)

In [8]:
#confirm pids match
(comb['pid_x'] != comb['pid_y']).sum()

0

In [9]:
#drop redundant pid column
comb = comb.rename(columns={'pid_x': 'pid'})
comb = comb.drop('pid_y', axis = 1)

In [10]:
#how merge_asof works: essentially a left join
right = pd.DataFrame({'a': [1, 5, 10], 'left_val': ['a', 'b', 'c']})
left = pd.DataFrame({'a': [1, 2, 3, 6, 7],
                      'right_val': [1, 2, 3, 6, 7]})
pd.merge_asof(left, right, on='a', direction = 'nearest')

,a,right_val,left_val
0,1,1,a
1,2,2,a
2,3,3,a
3,6,6,b
4,7,7,b


In [16]:
comb = comb.reset_index().drop('index',axis = 1)

In [18]:
comb.to_pickle('./joined.pkl')

In [7]:
df = pd.read_pickle('joined.pkl')

In [8]:
df.head()

,time,pid,x,y,z,TAC_Reading
1,1493739732016,JB3156,-2.556815,8.312009,14.090558,0.035549
2,1493739732035,JB3156,-2.499538,8.636412,13.906186,0.035549
3,1493739732055,JB3156,-2.471458,8.687223,13.677473,0.035549
4,1493739732075,JB3156,-2.244561,8.832140,13.288850,0.035549
5,1493739732095,JB3156,-2.453742,9.053032,12.610379,0.035549
